In [31]:
import numpy as np
from numpy.fft import fft,ifft

#creating the most basic dft code. O(N**2) time

def dft(x):
    N=len(x) #length of x
    k,n=np.meshgrid(np.arange(N),np.arange(N)) #meshgrid of wave numbers and space points
    M=np.exp(-1j*2*np.pi*k*n/N) #dft matrix
    F=np.matmul(M,x) #the fourier transform
    return F

    

In [32]:
#checking correctness of code

m=(2**10)*9 #size of input
m=int(m)

a=np.random.rand(m) #random input
b=dft(a) #created dft applied to a
c=fft(a) #in-built fft of numpy applied to a
e=np.max(np.abs(b-c)) #error
print(e)


4.065651674084134e-09


In [33]:
#comparing speed

%timeit fft
%timeit dft

#pretty close.surprising
#maybe my understanding of using time-it is off
'''I was following the link https://jakevdp.github.io/blog/2013/08/28/understanding-the-fft/.
There they mentioned there is an appreciable difference'''

8.36 ns ± 0.0173 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)
8.36 ns ± 0.0139 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


'I was following the link https://jakevdp.github.io/blog/2013/08/28/understanding-the-fft/.\nThere they mentioned there is an appreciable difference'

In [34]:
#divide and conquer algorithm. O(NlogN) time

def fastFT(x):
    N=len(x)
    while N%2==0:
        xe=x[0:N:2] #even indexed entries
        xo=x[1:N:2] #odd indexed entries
        fe=fastFT(xe) #fourier transform of even half
        fo=fastFT(xo) #fourier transform of odd half
        k=np.exp(-1j*2*np.pi*np.arange(N)/N) #the factor
        F1=fe+k[:int(N/2)]*fo #for less than N/2 part
        F2=fe+k[int(N/2):]*fo #for greater than N/2 part
        F=np.concatenate([F1,F2]) #joining the two parts to get the final fft
        return F
    else:
        return dft(x)
    

In [35]:
#checking correctness of fastFT

m=(2**10)*9
a=np.random.rand(m)
b=fastFT(a)
c=fft(a)
error=np.max(np.abs(b-c))
print(error)

#much better error

3.913339617472021e-12


In [36]:
#checking speed

%timeit fastFT
%timeit fft

#very close. But not much changed 
#maybe my understanding of using time-it is off

8.35 ns ± 0.00644 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)
8.35 ns ± 0.0127 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [37]:
''' 2/3 dealiasing for burgers eqn ut+uux=nu*uxx'''
from numpy.fft import fft,ifft,fftfreq


In [38]:
#writing the get-rhs fn of burgers without dealias filter
#parameters -function u,viscosity nu,wavenumber array k

def getrhs(u,nu,k):
    uk=fft(u) #discrete fourier transform of u
    uxk=1j*k*uk #discrete fourier transform of ux
    uxxk=-(k**2)*uk #discrete fourier transform of uxx
    return -u*np.real(ifft(uxk))+nu*np.real(ifft(uxxk))



In [39]:
#writing a get-rhs fn of burgers with dealias filter
#parameters -function u,viscosity nu,wavenumber array k,dealias filer dealias
#referred to https://www.uni-muenster.de/imperia/md/content/physik_tp/lectures/ss2017/numerische_Methoden_fuer_komplexe_Systeme_II/pseudospectral.pdf

def getrhsde(u,nu,k,dealias):
    uk=fft(u) #fft of u
    ukde=dealias*uk #dealiased uk
    ude=np.real(ifft(ukde)) #dealiased u
    uxk=1j*k*uk #discrete fourier transform of ux
    uxkde=dealias*uxk #dealiased fft of ux
    uxde=np.real(ifft(uxkde)) #dealiased first space derivative of u
    uxxk=-(k**2)*uk #discrete fourier transform of uxx
    nlp=ude*uxde #non-linear term with dealiased u
    nlpk=fft(nlp) #fft of non-linear term
    nlpkde=dealias*nlpk #dealiased fft of nl
    nlpde=np.real(ifft(nlpkde)) #dealiased nlp
    return -nlpde+nu*np.real(ifft(uxxk))
    




In [52]:
#trying errors for without dealiasing

Narray=np.array([2**7,2**9,2**11,2**12,2**13]) #space points array



for N in Narray:
    
    dx=2*np.pi/N
    x=np.arange(0,2*np.pi,dx)
    k=fftfreq(N,1/N)
    u=np.sin(x)
    nu=1
    computedrhs=getrhs(u,nu,k)
    actualrhs=-np.sin(x)*np.cos(x)-nu*np.sin(x)
    error=np.max(np.abs(computedrhs-actualrhs))
    print(error)
    


#Thank god,it works

6.384892614619275e-13
1.4445094426163152e-11
2.296302037407827e-10
8.230363612860003e-10
4.142286180730537e-09


In [53]:
#trying errors with dealias filter

#trying errors for without dealiasing

Narray=np.array([2**7,2**9,2**11,2**12,2**13]) #space points array



for N in Narray:

    dx=2*np.pi/N
    x=np.arange(0,2*np.pi,dx)
    k=fftfreq(N,1/N)
    u=np.sin(x)
    nu=1
    dealias=np.abs(k)<N/3
    computedrhs=getrhsde(u,nu,k,dealias)
    actualrhs=-np.sin(x)*np.cos(x)-nu*np.sin(x)
    error=np.max(np.abs(computedrhs-actualrhs))
    print(error)
    


#Thank god,it works
#pretty much same

6.373790384373024e-13
1.4445295654086365e-11
2.296423051717511e-10
8.2305212645295e-10
4.142344522950481e-09
